In [1]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from setup import WindowGenerator, compile_and_fit

file_array = os.listdir('../training_datasets')

def build(model):
    model_record = {}
    for i in range(len(file_array)):
        print(f'Run {i}')
        df = pd.read_csv("../training_datasets/"+file_array[i])

        time = pd.to_numeric(df.pop('time'))

        plot_cols = ['theta', 'thetadot', 'x', 'xdot']
        plot_features = df[plot_cols]

        column_indices = {name: i for i, name in enumerate(df.columns)}

        #splitting data by 70% training, 20% validating, 10% testing
        n = len(df)
        train_df = df[0:int(n*0.7)]
        val_df = df[int(n*0.7):int(n*0.9)]
        test_df = df[int(n*0.9):]

        num_features = df.shape[1]

        #normalize data: subtract the mean and divide by the standard deviation of each feature.
        train_mean = train_df.mean()
        train_std = train_df.std()

        train_df = (train_df - train_mean) / train_std
        val_df = (val_df - train_mean) / train_std
        test_df = (test_df - train_mean) / train_std

        df_std = (df - train_mean) / train_std
        df_std = df_std.melt(var_name='Column', value_name='Normalized')

        window = WindowGenerator(input_width=100, label_width=100, shift=1, train_df=train_df, val_df=val_df, test_df=test_df)
        compile_and_fit(model, window)
        model_record[i] = model.evaluate(window.test)

    return model_record

dense = tf.keras.models.load_model('../models/ssm_dense')
model_record = build(dense)

print(model_record.items() + len(model_record))

for name, value in model_record.items():
  print(f'Run {name}: {value[0]:0.5e}, {value[1]:0.5f}, {value[2]:0.2f}%')

<ipython-input-1-eb7b874ead38>:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd





Run 0
Epoch 1/20

109/109 [==============================] - 2s 7ms/step - loss: 3.9142e-04 - mean_absolute_error: 0.0109 - mean_absolute_percentage_error: 50.9153 - val_loss: 1.9589e-05 - val_mean_absolute_error: 0.0030 - val_mean_absolute_percentage_error: 4.6099
Epoch 2/20
109/109 [==============================] - 1s 7ms/step - loss: 5.0142e-04 - mean_absolute_error: 0.0090 - mean_absolute_percentage_error: 29.7021 - val_loss: 1.1973e-05 - val_mean_absolute_error: 0.0020 - val_mean_absolute_percentage_error: 2.6347
Epoch 3/20
109/109 [==============================] - 1s 7ms/step - loss: 5.0922e-05 - mean_absolute_error: 0.0043 - mean_absolute_percentage_error: 14.1615 - val_loss: 6.5295e-06 - val_mean_absolute_error: 0.0022 - val_mean_absolute_percentage_error: 3.7316
Epoch 4/20
109/109 [==============================] - 1s 7ms/step - loss: 2.8273e-05 - mean_absolute_error: 0.0033 - mean_absolute_percentage_error: 11.7283 - val_loss: 6.2315e-06 - val_mean_absolute_error: 0.0019

TypeError: unsupported operand type(s) for +: 'dict_items' and 'int'

In [2]:
print(model_record.items())

for name, value in model_record.items():
  print(f'Run {name}: {value[0]:0.5e}, {value[1]:0.5f}, {value[2]:0.2f}%')

dict_items([(0, [3.3866185731312726e-06, 0.0015005742898210883, 3.5778753757476807]), (1, [0.00027821699040941894, 0.013801191933453083, 23.62852668762207]), (2, [2.8651285902014934e-05, 0.004089881666004658, 7.404294013977051]), (3, [3.230630682082847e-05, 0.005286019761115313, 6.683470726013184]), (4, [0.00015949651424307376, 0.01187779288738966, 16.636173248291016])])
Run 0: 3.38662e-06, 0.00150, 3.58%
Run 1: 2.78217e-04, 0.01380, 23.63%
Run 2: 2.86513e-05, 0.00409, 7.40%
Run 3: 3.23063e-05, 0.00529, 6.68%
Run 4: 1.59497e-04, 0.01188, 16.64%


In [3]:
dense = tf.keras.models.load_model('../models/ssm_dense')
records['Dense'] = build(dense)

print(records.items())

for name, value in records.items():
  print(f'Run {name}: {value[0]:0.5e}, {value[1]:0.5f}, {value[2]:0.2f}%')

Run 0
Epoch 1/20
109/109 [==============================] - 2s 8ms/step - loss: 3.5431e-04 - mean_absolute_error: 0.0108 - mean_absolute_percentage_error: 50.9064 - val_loss: 1.3266e-05 - val_mean_absolute_error: 0.0028 - val_mean_absolute_percentage_error: 3.0032
Epoch 2/20
109/109 [==============================] - 1s 6ms/step - loss: 1.6301e-04 - mean_absolute_error: 0.0071 - mean_absolute_percentage_error: 33.1968 - val_loss: 3.9062e-06 - val_mean_absolute_error: 0.0018 - val_mean_absolute_percentage_error: 2.7395
Epoch 3/20
109/109 [==============================] - 1s 8ms/step - loss: 1.9794e-04 - mean_absolute_error: 0.0064 - mean_absolute_percentage_error: 34.1158 - val_loss: 3.3145e-05 - val_mean_absolute_error: 0.0039 - val_mean_absolute_percentage_error: 5.6330
Epoch 4/20
13/13 [==============================] - 0s 3ms/step - loss: 1.2468e-05 - mean_absolute_error: 0.0032 - mean_absolute_percentage_error: 9.2769
Run 1
Epoch 1/20
71/71 [==============================] - 2s 10

KeyboardInterrupt: 

In [4]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from setup import WindowGenerator, compile_and_fit

file_array = os.listdir('../training_datasets')
records = {}
def build(model):
    model_record = {}
    for i in range(len(file_array)):
        print(f'Run {i}')
        df = pd.read_csv("../training_datasets/"+file_array[i])

        time = pd.to_numeric(df.pop('time'))

        plot_cols = ['theta', 'thetadot', 'x', 'xdot']
        plot_features = df[plot_cols]

        column_indices = {name: i for i, name in enumerate(df.columns)}

        #splitting data by 70% training, 20% validating, 10% testing
        n = len(df)
        train_df = df[0:int(n*0.7)]
        val_df = df[int(n*0.7):int(n*0.9)]
        test_df = df[int(n*0.9):]

        num_features = df.shape[1]

        #normalize data: subtract the mean and divide by the standard deviation of each feature.
        train_mean = train_df.mean()
        train_std = train_df.std()

        train_df = (train_df - train_mean) / train_std
        val_df = (val_df - train_mean) / train_std
        test_df = (test_df - train_mean) / train_std

        df_std = (df - train_mean) / train_std
        df_std = df_std.melt(var_name='Column', value_name='Normalized')

        window = WindowGenerator(input_width=100, label_width=100, shift=1, train_df=train_df, val_df=val_df, test_df=test_df)
        compile_and_fit(model, window)
        model_record['val'][i] = model.evaluate(window.val)
        model_record['test'][i] = model.evaluate(window.test, verbose=0)
    return model_record

dense = tf.keras.models.load_model('../models/ssm_dense')
records['Dense'] = build(dense)

lstm = tf.keras.models.load_model('../models/ssm_lstm')
records['LSTM'] = build(lstm)

residual_lstm = tf.keras.models.load_model('../models/ssm_residual')
records['Residual LSTM'] = build(residual_lstm)

print(records.items())

for name, value in records.items():
  print(f'Run {name}: {value[0]:0.5e}, {value[1]:0.5f}, {value[2]:0.2f}%')

Run 0
Epoch 1/20
109/109 [==============================] - 2s 8ms/step - loss: 2.4290e-04 - mean_absolute_error: 0.0084 - mean_absolute_percentage_error: 30.3771 - val_loss: 1.1616e-05 - val_mean_absolute_error: 0.0026 - val_mean_absolute_percentage_error: 3.9390
Epoch 2/20
109/109 [==============================] - 1s 7ms/step - loss: 4.2389e-04 - mean_absolute_error: 0.0082 - mean_absolute_percentage_error: 42.5159 - val_loss: 1.7905e-05 - val_mean_absolute_error: 0.0041 - val_mean_absolute_percentage_error: 8.1505
Epoch 3/20
109/109 [==============================] - 1s 6ms/step - loss: 1.3819e-04 - mean_absolute_error: 0.0060 - mean_absolute_percentage_error: 31.0096 - val_loss: 1.1330e-05 - val_mean_absolute_error: 0.0026 - val_mean_absolute_percentage_error: 2.6619
Epoch 4/20
109/109 [==============================] - 1s 8ms/step - loss: 4.8048e-05 - mean_absolute_error: 0.0041 - mean_absolute_percentage_error: 21.8749 - val_loss: 1.3143e-05 - val_mean_absolute_error: 0.0035 - v

KeyError: 'val'

In [5]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from setup import WindowGenerator, compile_and_fit

file_array = os.listdir('../training_datasets')
records = {}
def build(model):
    model_record = {}
    for i in range(len(file_array)):
        print(f'Run {i}')
        df = pd.read_csv("../training_datasets/"+file_array[i])

        time = pd.to_numeric(df.pop('time'))

        plot_cols = ['theta', 'thetadot', 'x', 'xdot']
        plot_features = df[plot_cols]

        column_indices = {name: i for i, name in enumerate(df.columns)}

        #splitting data by 70% training, 20% validating, 10% testing
        n = len(df)
        train_df = df[0:int(n*0.7)]
        val_df = df[int(n*0.7):int(n*0.9)]
        test_df = df[int(n*0.9):]

        num_features = df.shape[1]

        #normalize data: subtract the mean and divide by the standard deviation of each feature.
        train_mean = train_df.mean()
        train_std = train_df.std()

        train_df = (train_df - train_mean) / train_std
        val_df = (val_df - train_mean) / train_std
        test_df = (test_df - train_mean) / train_std

        df_std = (df - train_mean) / train_std
        df_std = df_std.melt(var_name='Column', value_name='Normalized')

        window = WindowGenerator(input_width=100, label_width=100, shift=1, train_df=train_df, val_df=val_df, test_df=test_df)
        compile_and_fit(model, window)
        model_record[0][i] = model.evaluate(window.val)
        model_record[1][i] = model.evaluate(window.test, verbose=0)
    return model_record

dense = tf.keras.models.load_model('../models/ssm_dense')
records['Dense'] = build(dense)

lstm = tf.keras.models.load_model('../models/ssm_lstm')
records['LSTM'] = build(lstm)

residual_lstm = tf.keras.models.load_model('../models/ssm_residual')
records['Residual LSTM'] = build(residual_lstm)

print(records.items())

for name, value in records.items():
  print(f'Run {name}: {value[0]:0.5e}, {value[1]:0.5f}, {value[2]:0.2f}%')

Run 0
Epoch 1/20
109/109 [==============================] - 2s 9ms/step - loss: 3.4295e-04 - mean_absolute_error: 0.0105 - mean_absolute_percentage_error: 64.4914 - val_loss: 4.3067e-05 - val_mean_absolute_error: 0.0055 - val_mean_absolute_percentage_error: 14.2891
Epoch 2/20
109/109 [==============================] - 1s 7ms/step - loss: 1.2241e-04 - mean_absolute_error: 0.0061 - mean_absolute_percentage_error: 34.0699 - val_loss: 8.0267e-06 - val_mean_absolute_error: 0.0026 - val_mean_absolute_percentage_error: 5.5979
Epoch 3/20
109/109 [==============================] - 1s 6ms/step - loss: 4.1162e-04 - mean_absolute_error: 0.0099 - mean_absolute_percentage_error: 43.8977 - val_loss: 5.3617e-05 - val_mean_absolute_error: 0.0071 - val_mean_absolute_percentage_error: 12.4892
Epoch 4/20
109/109 [==============================] - 1s 7ms/step - loss: 2.9195e-04 - mean_absolute_error: 0.0077 - mean_absolute_percentage_error: 59.5118 - val_loss: 7.1937e-07 - val_mean_absolute_error: 6.3451e-

KeyError: 0

In [6]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from setup import WindowGenerator, compile_and_fit

file_array = os.listdir('../training_datasets')
records = {}
def build(model):
    model_record = {}
    for i in range(len(file_array)):
        print(f'Run {i}')
        df = pd.read_csv("../training_datasets/"+file_array[i])

        time = pd.to_numeric(df.pop('time'))

        plot_cols = ['theta', 'thetadot', 'x', 'xdot']
        plot_features = df[plot_cols]

        column_indices = {name: i for i, name in enumerate(df.columns)}

        #splitting data by 70% training, 20% validating, 10% testing
        n = len(df)
        train_df = df[0:int(n*0.7)]
        val_df = df[int(n*0.7):int(n*0.9)]
        test_df = df[int(n*0.9):]

        num_features = df.shape[1]

        #normalize data: subtract the mean and divide by the standard deviation of each feature.
        train_mean = train_df.mean()
        train_std = train_df.std()

        train_df = (train_df - train_mean) / train_std
        val_df = (val_df - train_mean) / train_std
        test_df = (test_df - train_mean) / train_std

        df_std = (df - train_mean) / train_std
        df_std = df_std.melt(var_name='Column', value_name='Normalized')

        window = WindowGenerator(input_width=100, label_width=100, shift=1, train_df=train_df, val_df=val_df, test_df=test_df)
        compile_and_fit(model, window)
        model_record[i] = [model.evaluate(window.val), model.evaluate(window.test, verbose=0)]
    return model_record

dense = tf.keras.models.load_model('../models/ssm_dense')
records['Dense'] = build(dense)

lstm = tf.keras.models.load_model('../models/ssm_lstm')
records['LSTM'] = build(lstm)

residual_lstm = tf.keras.models.load_model('../models/ssm_residual')
records['Residual LSTM'] = build(residual_lstm)

print(records.items())

for name, value in records.items():
  print(f'Run {name}: {value[0]:0.5e}, {value[1]:0.5f}, {value[2]:0.2f}%')

Run 0
Epoch 1/20
109/109 [==============================] - 2s 7ms/step - loss: 3.3231e-04 - mean_absolute_error: 0.0100 - mean_absolute_percentage_error: 44.3882 - val_loss: 1.0980e-05 - val_mean_absolute_error: 0.0028 - val_mean_absolute_percentage_error: 6.9452
Epoch 2/20
109/109 [==============================] - 1s 7ms/step - loss: 1.2548e-04 - mean_absolute_error: 0.0060 - mean_absolute_percentage_error: 33.0678 - val_loss: 5.0989e-06 - val_mean_absolute_error: 0.0018 - val_mean_absolute_percentage_error: 2.7703
Epoch 3/20
109/109 [==============================] - 1s 7ms/step - loss: 1.6128e-04 - mean_absolute_error: 0.0059 - mean_absolute_percentage_error: 20.2170 - val_loss: 3.1207e-05 - val_mean_absolute_error: 0.0052 - val_mean_absolute_percentage_error: 9.0966
Epoch 4/20
29/29 [==============================] - 0s 2ms/step - loss: 1.4844e-05 - mean_absolute_error: 0.0033 - mean_absolute_percentage_error: 8.2810
Run 1
Epoch 1/20
71/71 [==============================] - 1s 9m

TypeError: unsupported format string passed to list.__format__

In [7]:
print(records.items())

for name, value in records.items():
  print(f'Run {name}: {value}')

dict_items([('Dense', {0: [[1.4843547432974447e-05, 0.003268602304160595, 8.281023979187012], [1.3249820767669007e-05, 0.002897998783737421, 9.23784351348877]], 1: [[0.00013962446246296167, 0.009107818827033043, 14.751562118530273], [0.00011147221084684134, 0.008421461097896099, 11.253528594970703]], 2: [[3.7696005165344104e-05, 0.004389045760035515, 2.864558219909668], [1.9161574527970515e-05, 0.0031013842672109604, 2.0168545246124268]], 3: [[0.00013609346933662891, 0.008805079385638237, 39.09708786010742], [0.0001342993346042931, 0.008303673006594181, 8.536376953125]], 4: [[2.3346179659711197e-05, 0.00407529016956687, 12.143778800964355], [5.5716045608278364e-05, 0.005758261773735285, 9.40739917755127]]}), ('LSTM', {0: [[1.854750189522747e-05, 0.0035322289913892746, 7.8758463859558105], [1.8703791283769533e-05, 0.003578786039724946, 9.185182571411133]], 1: [[1.3238482097222004e-05, 0.002101586665958166, 8.741935729980469], [1.100067311199382e-05, 0.0018856287933886051, 6.221638202667

In [8]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from setup import WindowGenerator, compile_and_fit

file_array = os.listdir('../training_datasets')
records = {}
def build(model):
    model_record = {}
    for i in range(len(file_array)):
        print(f'Run {i}')
        df = pd.read_csv("../training_datasets/"+file_array[i])

        time = pd.to_numeric(df.pop('time'))

        plot_cols = ['theta', 'thetadot', 'x', 'xdot']
        plot_features = df[plot_cols]

        column_indices = {name: i for i, name in enumerate(df.columns)}

        #splitting data by 70% training, 20% validating, 10% testing
        n = len(df)
        train_df = df[0:int(n*0.7)]
        val_df = df[int(n*0.7):int(n*0.9)]
        test_df = df[int(n*0.9):]

        num_features = df.shape[1]

        #normalize data: subtract the mean and divide by the standard deviation of each feature.
        train_mean = train_df.mean()
        train_std = train_df.std()

        train_df = (train_df - train_mean) / train_std
        val_df = (val_df - train_mean) / train_std
        test_df = (test_df - train_mean) / train_std

        df_std = (df - train_mean) / train_std
        df_std = df_std.melt(var_name='Column', value_name='Normalized')

        window = WindowGenerator(input_width=100, label_width=100, shift=1, train_df=train_df, val_df=val_df, test_df=test_df)
        compile_and_fit(model, window)
        model_record[i] = [model.evaluate(window.val), model.evaluate(window.test, verbose=0)]
    return model_record

dense = tf.keras.models.load_model('../models/ssm_dense')
records['Dense'] = build(dense)

lstm = tf.keras.models.load_model('../models/ssm_lstm')
records['LSTM'] = build(lstm)

residual_lstm = tf.keras.models.load_model('../models/ssm_residual')
records['Residual LSTM'] = build(residual_lstm)

print(records.items())

for name, item in records.items():
  print(f'Run {name}: {len(item)}')

Run 0
Epoch 1/20
109/109 [==============================] - 1s 5ms/step - loss: 3.4333e-04 - mean_absolute_error: 0.0094 - mean_absolute_percentage_error: 41.0716 - val_loss: 1.4883e-05 - val_mean_absolute_error: 0.0031 - val_mean_absolute_percentage_error: 6.2492
Epoch 2/20
109/109 [==============================] - 0s 3ms/step - loss: 7.1344e-05 - mean_absolute_error: 0.0052 - mean_absolute_percentage_error: 20.1567 - val_loss: 4.6816e-06 - val_mean_absolute_error: 0.0019 - val_mean_absolute_percentage_error: 4.3397
Epoch 3/20
109/109 [==============================] - 0s 3ms/step - loss: 3.9403e-05 - mean_absolute_error: 0.0039 - mean_absolute_percentage_error: 17.3634 - val_loss: 1.7236e-06 - val_mean_absolute_error: 9.5368e-04 - val_mean_absolute_percentage_error: 2.9964
Epoch 4/20
109/109 [==============================] - 0s 3ms/step - loss: 2.6578e-04 - mean_absolute_error: 0.0067 - mean_absolute_percentage_error: 33.2291 - val_loss: 4.0607e-05 - val_mean_absolute_error: 0.0049

KeyboardInterrupt: 

In [9]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from setup import WindowGenerator, compile_and_fit

file_array = os.listdir('../training_datasets')
records = {}
def build(model):
    model_record = {}
    for i in range(len(file_array)):
        print(f'Run {i}')
        df = pd.read_csv("../training_datasets/"+file_array[i])

        time = pd.to_numeric(df.pop('time'))

        plot_cols = ['theta', 'thetadot', 'x', 'xdot']
        plot_features = df[plot_cols]

        column_indices = {name: i for i, name in enumerate(df.columns)}

        #splitting data by 70% training, 20% validating, 10% testing
        n = len(df)
        train_df = df[0:int(n*0.7)]
        val_df = df[int(n*0.7):int(n*0.9)]
        test_df = df[int(n*0.9):]

        num_features = df.shape[1]

        #normalize data: subtract the mean and divide by the standard deviation of each feature.
        train_mean = train_df.mean()
        train_std = train_df.std()

        train_df = (train_df - train_mean) / train_std
        val_df = (val_df - train_mean) / train_std
        test_df = (test_df - train_mean) / train_std

        df_std = (df - train_mean) / train_std
        df_std = df_std.melt(var_name='Column', value_name='Normalized')

        window = WindowGenerator(input_width=100, label_width=100, shift=1, train_df=train_df, val_df=val_df, test_df=test_df)
        compile_and_fit(model, window)
        model_record[i] = [model.evaluate(window.val), model.evaluate(window.test, verbose=0)]
    return model_record

dense = tf.keras.models.load_model('../models/ssm_dense')
records['Dense'] = build(dense)

lstm = tf.keras.models.load_model('../models/ssm_lstm')
records['LSTM'] = build(lstm)

residual_lstm = tf.keras.models.load_model('../models/ssm_residual')
records['Residual LSTM'] = build(residual_lstm)

print(records.items())

for name, item in records.items():
  print(f'Run {name}: {item[4][0][2]}, {item[4][1][2]}')

Run 0
Epoch 1/20
109/109 [==============================] - 1s 5ms/step - loss: 3.9364e-04 - mean_absolute_error: 0.0100 - mean_absolute_percentage_error: 45.7330 - val_loss: 1.8622e-05 - val_mean_absolute_error: 0.0028 - val_mean_absolute_percentage_error: 3.0393
Epoch 2/20
109/109 [==============================] - 0s 3ms/step - loss: 7.2281e-05 - mean_absolute_error: 0.0051 - mean_absolute_percentage_error: 18.4200 - val_loss: 1.4258e-05 - val_mean_absolute_error: 0.0034 - val_mean_absolute_percentage_error: 4.5802
Epoch 3/20
109/109 [==============================] - 0s 3ms/step - loss: 1.0524e-04 - mean_absolute_error: 0.0056 - mean_absolute_percentage_error: 32.6275 - val_loss: 1.5780e-05 - val_mean_absolute_error: 0.0035 - val_mean_absolute_percentage_error: 7.3623
Epoch 4/20
29/29 [==============================] - 0s 2ms/step - loss: 9.5870e-05 - mean_absolute_error: 0.0093 - mean_absolute_percentage_error: 15.7281
Run 1
Epoch 1/20
71/71 [==============================] - 1s 5